## BERT Transformer

In [12]:
# Stuff that makes the notebook look nicer
import warnings
warnings.filterwarnings('ignore')
import logging
from tensorflow import get_logger
get_logger().setLevel(logging.ERROR)

# Load Huggingface transformers
from transformers import TFBertModel, BertConfig, BertTokenizerFast
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

# Libraries to import and process the data set
from Utilities import directories
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime

# Visualization
import matplotlib.pyplot as plt

from keras.utils.vis_utils import plot_model
import pathlib

In [13]:
# Set file directories
model_name = "InitialTestModel"
read_dir = pathlib.Path.joinpath(directories.data, "example_extract.txt")
save_dir = pathlib.Path.joinpath(directories.savedModels, model_name)


In [14]:
### --------- Load and transform data ---------- ###
with open(read_dir, "rb") as file:   # Unpickling
    data = pickle.load(file)

# Transfer data to pandas ds and set model output (classes) as categorical
data_df = pd.DataFrame(data, index=['Description', 'CPC_label']).transpose()
data_df['CPC_label'] = pd.Categorical(data_df['CPC_label'])
# Transform output to numeric
data_df['CPC_code'] = data_df['CPC_label'].cat.codes
# Split into train and test
train, test = train_test_split(data_df, test_size = 0.1)
print(len(train), len(test))

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\MLData\\thesis\\Datasets\\example_extract.txt'

In [ ]:
test

In [ ]:
### --------- Setup BERT ---------- ###
# Name of the BERT model to use
model_name = 'bert-base-uncased'
# Max length of tokens
max_length = 100
# Load transformers config and set output_hidden_states to False
# from_pretrained loads weights from pretrained model
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

In [ ]:
# View default configurations
config

In [ ]:
### ------- Build the model ------- ###
# Load the MainLayer
bert = transformer_model.layers[0]

# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)

# Then build your model output
cpc_class = Dense(units=len(data_df.CPC_code.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='cpc')(pooled_output)
outputs = {'cpc': cpc_class}

# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

# Take a look at the model
with open(save_dir + '/summary.txt', 'w') as file:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: file.write(x + '\n'))

plot_model(model, to_file="test.png")

In [ ]:
bert[1]

In [ ]:
### ------- Set hyperparameters and input and output ------- ###
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

# Set loss and metrics
loss = {'cpc': CategoricalCrossentropy(from_logits = True)} # Using from_logits=true means that the prediction tensor is
                                                            # one hot encoded. By default, it expects a probability
                                                            # distribution
metric = {'cpc': CategoricalAccuracy('accuracy')}

# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = metric)

# Prepare output data for the model
y_cpc = to_categorical(train['CPC_code'], num_classes=16)

# Tokenize the input (takes some time)
x = tokenizer(
    text=train['Description'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

In [ ]:
# One hot vector of 38 sequences x 16 classes
print(y_cpc)

# 38 sequences x 100 token ids because of max_length=100
print(x['input_ids'].shape)

In [ ]:
### ------- Train ------- ###
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'cpc': y_cpc},
    validation_split=0.1,
    batch_size=32,
    epochs=10)

In [ ]:
### ----- Evaluate the model ------ ####
# Evaluate test data
y_cpc_test = to_categorical(test['CPC_code'], num_classes=16)
test_x = tokenizer(
    text=test['Description'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)# Run evaluation
model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'cpc': y_cpc_test}
)

In [ ]:
### ----- Visualize training history ------ ####
# summarize history for accuracy
fig1 = plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.tight_layout()
plt.show()
# summarize history for loss
fig2 = plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
### ----- Save the model and figures to disk ------ ####
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
fig1.savefig(save_dir + "/accuracy_" + timestamp, dpi=300)
fig2.savefig(save_dir + "/loss_" + timestamp, dpi=300)
model.save(save_dir + "/" + timestamp)

Adapted from:
https://towardsdatascience.com/multi-label-multi-class-text-classification-with-bert-transformer-and-keras-c6355eccb63a
